## Timetabling Program for St. Margaret's School (ILP Only)

In [22]:
"""This file generates the mathematically optimal timetable for St. Margaret's School.
As in the case of Lalaland School, we find the optimal schedule by setting up the problem 
as an Integer Linear Program which we solve thereafter using the Coin-or branch and cut (CBC)
solver."""


# Install ortools module
import sys
!{sys.executable} -m pip install ortools 

# Import libraries
import time
import numpy as np
import pandas as pd

# Import the Google OR-Tools linear solver wrapper
from ortools.linear_solver import pywraplp


#________________________________________________________________________________________
# STEP 1 - READ IN DATA
# Import .xlsx file containing a table of students' preferred courses
StudentInput = pd.ExcelFile("SMS_student_data.xlsx") 
StudentMatrix = pd.read_excel(StudentInput, 'Data') 
StudentInfo = StudentMatrix.values.tolist()
StudentMatrix[1:5]

# Import .xlsx file containing the course data
CourseInput = pd.ExcelFile("SMS_course_data.xlsx") 
CourseMatrix = pd.read_excel(CourseInput, 'Data') 
CourseInfo = CourseMatrix.values.tolist()
CourseMatrix[1:5]


#________________________________________________________________________________________
# STEP 2 - FORMAT DATA
# Partition the offered courses into ShortCourses (ABC) and LongCourses (DEFGHI).
# We only consider the CourseID in the course data file, and we disregard 
# repeated sections of the course.
ShortCourses=[]
LongCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==1:
        if CourseInfo[m][3]=="YES":
            ShortCourses.append(CourseID)
        else:
            LongCourses.append(CourseID)
            
# Partition the offered courses into OneSection and TwoSection
AllCourses=[CourseInfo[m][1] for m in range(len(CourseInfo))]
OneSection=[]
TwoSections=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==2:
        TwoSections.append(CourseID)   
for CourseID in AllCourses:
    if not CourseID in TwoSections:
        OneSection.append(CourseID)
        
# Identify all Core Courses
CoreCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][6]=="YES" and CourseInfo[m][2]==1:
        CoreCourses.append(CourseID)  

# Determine the preference coefficients based on the student data.
# We have 10 points for a Core Course, 3 points for a Grade 12 elective
# and 1 point for a Grade 11 elective.

n = len(StudentInfo)
m = 100
P = np.zeros((n,m), dtype=int)
for i in range(n):
    for j in range(m):
        P[i,j]=-1

MaxScore=0
for i in range(n):
    for j in range(2,12):
        StudentGrade = StudentInfo[i][1]
        CoursePick = StudentInfo[i][j]
        if CoursePick > 0:
            if CoursePick in CoreCourses: P[i,CoursePick]=10
            else:
                if StudentGrade==12: P[i,CoursePick]=3
                if StudentGrade==11: P[i,CoursePick]=1

                    
#________________________________________________________________________________________
# STEP 3 - OPTIMIZE TIMETABLE FOR SHORT BLOCKS
# Optimize short blocks
solver = pywraplp.Solver('St. Margarets School', 
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
ShortBlocks = [1,2,3]

# Define boolean variables
x = {}
for s in Sections:
    for j in ShortCourses:
        for k in ShortBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 

            
# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(ShortCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(ShortCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in ShortCourses:
    for k in ShortBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most eight classes per block
for k in ShortBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in ShortCourses) <= 8)
        
# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    ShortCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in ShortCourses:
            ShortCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(ShortCourseList)>1:
        for k in ShortBlocks:
            solver.Add(sum(x[ShortCourseList[j][0],ShortCourseList[j][1],k] 
                           for j in range(len(ShortCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in ShortBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in ShortBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in ShortCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in ShortCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in ShortBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(ShortCourses).intersection(TwoSections):
    for k in ShortBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        

solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents 
                           for j in ShortCourses for k in ShortBlocks))

# Call the solver and print summary statistics of short blocks schedule
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', 
      round(solver.Objective().Value()))

# Compute runtime
solving_time = time.time() - start_time
print('The code ran in', round(solving_time,1), 'seconds')


#________________________________________________________________________________________
# STEP 4 - OBTAIN OUTPUT TIMETABLE FOR SHORT BLOCKS
# Print the courses scheduled in each Short block and the number of
# students registered in each course.

ShortSchedule=[ [0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            if y[i,j,k].solution_value()==1: 
                ShortSchedule[i][k-1]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in ShortBlocks:
    print()
    print("Block", k, "courses are")

    for j in ShortCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students")
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students")

#________________________________________________________________________________________
# STEP 5 - OPTIMIZE TIMETABLE FOR LONG BLOCKS
# Optimize long blocks
solver = pywraplp.Solver('St. Margarets School', 
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
LongBlocks = [4,5,6,7,8,9]

# Define boolean variables
x = {}
for s in Sections:
    for j in LongCourses:
        for k in LongBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 

            
# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(LongCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(LongCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in LongCourses:
    for k in LongBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most six classes per block
for k in LongBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in LongCourses) <= 6)
        
# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    LongCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in LongCourses:
            LongCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(LongCourseList)>1:
        for k in LongBlocks:
            solver.Add(sum(x[LongCourseList[j][0],LongCourseList[j][1],k] 
                           for j in range(len(LongCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in LongBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in LongBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in LongCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in LongCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in LongBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(LongCourses).intersection(TwoSections):
    for k in LongBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        
            
# CONSTRAINT 11: A few extra constraints were coded in manually, could get rid of later
#solver.Add(x[1,70,7]+x[1,70,8]+x[1,70,9]==1)
#solver.Add(x[1,74,7]+x[1,74,8]+x[1,74,9]==1)
#solver.Add(x[1,55,4]+x[1,55,5]+x[1,55,6]==1)


solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents 
                           for j in LongCourses for k in LongBlocks))

# Call the solver and print summary statistics of long blocks schedule
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', 
      round(solver.Objective().Value()))

# Compute runtime
solving_time = time.time() - start_time
print('The code ran in', round(solving_time,1), 'seconds')


#________________________________________________________________________________________
# STEP 6 - OBTAIN OUTPUT TIMETABLE FOR LONG BLOCKS
# Print the courses scheduled in each Long block and the number of
# students registered in each course.
LongSchedule=[ [0,0,0,0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            if y[i,j,k].solution_value()==1: 
                LongSchedule[i][k-4]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in LongBlocks:
    print()
    print("Block", k, "courses are")

    for j in LongCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students")
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students")

            
#________________________________________________________________________________________
# STEP 6 - OBTAIN FINAL MASTER TIMETABLE 
# Print stats on the quality of our final timetable
CoreMax=0
Gr12Max=0
Gr11Max=0
CoreScore=0
Gr12Score=0
Gr11Score=0

for i in AllStudents:
    for j in range(m):
        if P[i,j]==10: 
            CoreMax+=1
            if j in ShortSchedule[i]: CoreScore+=1
            if j in LongSchedule[i]: CoreScore+=1
        if P[i,j]==3: 
            Gr12Max+=1
            if j in ShortSchedule[i]: Gr12Score+=1
            if j in LongSchedule[i]: Gr12Score+=1
        if P[i,j]==1: 
            Gr11Max+=1
            if j in ShortSchedule[i]: Gr11Score+=1
            if j in LongSchedule[i]: Gr11Score+=1
                
print()                
print("TOTAL: Students got into", CoreScore, "out of", CoreMax, "core courses")
print("Grade 12s got into", Gr12Score, "out of", Gr12Max, "elective courses")
print("Grade 11s got into", Gr11Score, "out of", Gr11Max, "elective courses")
print()

# Print list of all students who didn't get into ALL of their desired courses
# and how many courses that missed getting into.
for i in AllStudents:
    choices=0
    for j in range(2,12):
        if StudentInfo[i][j]>0:
            choices+=1
    for j in ShortSchedule[i]:
        if j>0:
            choices-=1
    for j in LongSchedule[i]:
        if j>0:
            choices-=1
    if choices>0:
        print("Student", i, "missed", choices, "-", 
              ShortSchedule[i],LongSchedule[i],StudentInfo[i])


Optimization Complete with Total Happiness Score of 1016
The code ran in 1.9 seconds

Block 1 courses are
Course 10 Section 2 with 0 students
Course 16 Section 1 with 2 students
Course 28 Section 1 with 13 students
Course 45 Section 1 with 14 students
Course 75 Section 1 with 6 students
Course 79 Section 1 with 6 students

Block 2 courses are
Course 10 Section 1 with 3 students
Course 12 Section 1 with 1 students
Course 28 Section 2 with 12 students
Course 29 Section 1 with 18 students
Course 76 Section 1 with 10 students
Course 80 Section 1 with 4 students

Block 3 courses are
Course 7 Section 1 with 4 students
Course 29 Section 2 with 15 students
Course 30 Section 1 with 7 students
Course 31 Section 1 with 14 students
Course 53 Section 1 with 1 students
Course 56 Section 1 with 1 students
Course 59 Section 1 with 3 students
Course 78 Section 1 with 5 students

Optimization Complete with Total Happiness Score of 1803
The code ran in 98.3 seconds

Block 4 courses are
Course 61 Section